In [1]:
#Creates a GUI window

import tkinter as tk

class Root:
    
    def __init__(self, rootType):
        if rootType == "parent":                               # Indicates if window is the main windows of an application
            self.__root = tk.Tk()                                  # If it is: using tkinter's TK() method 
        else:
            self.__root = tk.Toplevel()                            # Else: use tkinters Toplevel() method
            
    def config(self, title, geometry, ico, state, resizable):  # Used for setting up window configurations
        self.__root.geometry(geometry)                             # Size and position of window
        self.__root.iconbitmap(ico)                                # Add custom icon for window
        self.__root.state(state)                                   # This makes the window fullscreen (except for Winodws taskbar)
        self.__root.title(title)                                   # Title for window
        self.__root.resizable(resizable, resizable)                # Control if the window can be resizable
        return self
    
    def topmost(self, topmostValue):                           # Make window either topmost or not topmost
        self.__root.attributes("-topmost", topmostValue)
        return self                                                     
    
    def hide(self):                                            # Hide window
        self.__root.withdraw()
        return self                                        
    
    def show(self):                                            # Show window
        self.__root.deiconify()
        return self                                         
        
    def getRoot(self):                                         # Getter for root window
        return self.__root
    
    def execute(self):                                         # Run window loop
        self.__root.mainloop()

In [2]:
# Parent of all GUI Widget Types

import tkinter as tk

class Widget:
        
    def config(self, configDict):                          # This method takes a dict of configurations and adds it to the widget
        for config in configDict.keys():
            self._widget[config] = configDict[config]
        return self                                        
    
    def execute(self, fill, ipadx, ipady, side, expand):  # Adds widget to root window
        self._widget.pack(fill = fill,
                          ipadx = ipadx,
                          ipady = ipady,
                          side = side,
                          expand = expand)
        return self
    
    def getWidget(self):                                  # Getter for widget
        return self._widget

In [3]:
# Child of widget, wrapper class for Tkinter's Label class

import tkinter as tk

class Label(Widget):
    
    def __init__(self, root):
        self._widget = tk.Label(root)     # Constrcutor assigns widget var to tkinter's Label class object

In [4]:
# Child of widget, wrapper class for Tkinter's Button class

import tkinter as tk

class Button(Widget):
    
    def __init__(self, root):
        self._widget = tk.Button(root)    # Constrcutor assigns widget var to tkinter's Button class object

In [5]:
# Child of widget, wrapper class for Tkinter's Radiobutton class

import tkinter as tk

class RadioButton(Widget):
    
    def __init__(self, root):
        self._widget = tk.Radiobutton(root)    # Constrcutor assigns widget var to tkinter's Radiobutton class object

In [6]:
# Child of widget, parent of Spinbox & RadioButtonGroup classess

class EntryWidget(Widget):
    
    def __init__(self, root, config, default, name, parameterName):                               
        self._root = root                                                                             # Root window                                                                       
        self._config = config                                                                         # Dict of widget specific settings 
        self.__name = name                                                                            # Name of the tkinter StringVar that will hold user entry data
        self._config[parameterName] = tk.StringVar(self._root, name = self.__name, value = default)   # Bind this StringVar to the config dict
        
    def getValue(self):
        return self._root.getvar(name = self.__name)                                                  # Getter for StringVar

In [7]:
# Child of widget, wrapper class for Tkinter's Entry class

import tkinter as tk

class Entry(Widget):
    
    def __init__(self, root):
        self._widget = tk.Entry(root)   # Constrcutor assigns widget var to tkinter's Entry class object
        self._widget.focus_force()      # Force cursor to highlight input area
        
    def getInput(self):
        return self._widget.get()       # Getter for user input

In [8]:
# Child of widget, wrapper class for Tkinter's scrolledtext class and 

import tkinter.scrolledtext as scrolledtext

class Text(Widget):
    
    def __init__(self, root, document):                                           
        self.__document = document                                                   # document will be text to display in widget
        self._widget = scrolledtext.ScrolledText(root)                               # Assigns widget var to tkinter's ScrolledText widget from tkinters scrolledtext
        self._widget.insert("1.0", self.__document.getContent())                     # Add document to the widget
        self.__previousConfig = Config().setConfigFromFile().getConfigFromFile()     # Read config.json (which creates a dict file in memory, with the presets the program uses)
        self.config({"font" : ("Courier", self.__previousConfig["FontSize"])})       # Font size will be initially be based on config.json
        self._widget.tag_configure("center", justify = "center")                     # Create tag that will center the text displayed
        self._widget.tag_add("center", "1.0", "end")                                 # Add this tag to widget
        self.__selectedPoints = []                                                   # Hold coords of the first and last line selected
        self.__savedQuery = []                                                       # Holds lines where target query was found in widget
        self.__curentIndex = -1                                                       # Current line selected
    
    def highlightLine(self):                                                      # Method to highlight a specfic line
        if self._widget.tag_ranges("highlight"):                                     # If Line is already highlighted:                 
            self._widget.tag_delete("highlight")                                        # Remove highlight tag
        index = int(float(self._widget.index(tk.CURRENT)))                           # Get current line selected 
        self._widget.tag_configure("highlight",background = "#004145")               # Create highlight tag
        self._widget.tag_add("highlight", f"{index}.0", f"{index + 1}.0")            # Add highlight tag
        
    def makeSelection(self):                                                       # Based on where the user's mouse is, will make a selection of the first and last line of txt doc                                                  
        index = int(float(self._widget.index(tk.INSERT)))                             # Index of current mouse location 

        # If there is no current slection or if there is a current selection and the current mose location is diffrent from the last selection 
        if (len(self.__selectedPoints) == 0) or ((len(self.__selectedPoints) > 0) and (f"{index}.0" != self.__selectedPoints[-1][0])):    

            # Only two lines (beginning and end) can be selected. if it's a thrid point, the current selection resets
            if len(self.__selectedPoints) % 2 == 0:
                self._widget.tag_delete("selection")                              # Stop highlighting current lines that are selected
                self._widget.tag_delete("select")                                 # Stop highlighting current line selected

            self._widget.tag_configure("select",background="#97910F")             # Define the line selection color
            self._widget.tag_add("select", f"{index}.0", f"{index + 1}.0")        # Highlight selected line
            
            self.__selectedPoints.append((f"{index}.0", f"{index + 1}.0"))        # Add the the selection to a list
            
            if len(self.__selectedPoints) % 2 == 0:                               # If two lines are selected"
                self._widget.tag_configure("selection",background = "#97910F")        # Define the color for multiple line selection
                
                # If the first selected line is less than the current selected line (Forward)
                if index > int(float(self.__selectedPoints[-2][0])):                                          
                    self._widget.tag_add("selection", self.__selectedPoints[-2][0], self.__selectedPoints[-1][1])

                # If the first selected line is greater than or equal to the current selected line (Backward)
                else:
                    self._widget.tag_add("selection", self.__selectedPoints[-1][0], self.__selectedPoints[-2][1])
                     
    def __updateCurrentIndex(self, direction):                                          #Finds the desired word location and highlights it
        
        # If a word has already been found, reset the selection 
        if self._widget.tag_ranges("find"):
            self._widget.tag_remove("find", "1.0", "end")                                   #Remove highlighting of specfic word found
            
        self._widget.tag_configure("find", background = "#646464", foreground = "black")    # Define the highlight style for word found
        self.__curentIndex = ((self.__curentIndex + direction) % len(self.__savedQuery))    # Current selected word found (ex. 3 of 30 for "Hello Wolrd")
        self._widget.tag_add("find",                                                        # Name of tag
                             f"{self.__savedQuery[self.__curentIndex]}.0",                  # Begining index of word found
                             f"{self.__savedQuery[self.__curentIndex] + 1}.0")              # Ending index of word found
        self._widget.see(f"{self.__savedQuery[self.__curentIndex]}.0")                      # Focus user perspective on word found
        return f"{self.__curentIndex+1} of {len(self.__savedQuery)}"                        # Return the location of this word, in relation to all items found (ex. 3 of 30)
    
    def queryFile(self, target, direction):                                                 # This queries the text widget for desired word                                   
        self._widget.focus_force()                                                               # Focus user perspective on input
        if self.__savedQuery:                                                                    # If a word has been found or a query has already been made to the TextDocument object
            return self.__updateCurrentIndex(direction)                                              # Find the word location and highlight
        else:                                            
            self.__savedQuery = [index + 1 for index in self.__document.findTarget(target)] # If query has not already been made to the TextDocument object, do it using findTarget method
            
            if self.__savedQuery:                            # If word is found
                return self.__updateCurrentIndex(direction)       # Find the word location and highlight
            else:
                return "Not Found"                           # Word has not been found
            
    def resetSelectedPoints(self):              # Resets all tags related to class
        self.__selectedPoints = []                    # Reset list that holds selected points
        self.__savedQuery = []                        # Reset list that holds the locationms of a specific word found
        self.__curentIndex = -1                        # Reset the var that hold whihc line is currently selected
        self._widget.tag_delete("selection")          # Reset the tag for multiples lines selected
        self._widget.tag_delete("select")             # Reset the tag for a single line selected

In [9]:
# Child of EntryWidget, which is a child of Widget class. This is a wrapper class for Tkinter's Spinbox class

import tkinter as tk

class Spinbox(EntryWidget):
    
    def __init__(self, root, spinboxConfig, default, name, parameterName):  # Constuctor for Spinbox class
        super().__init__(root, spinboxConfig, default, name, parameterName)     # Constructor for EntryWidget class
        self._widget = tk.Spinbox(root)                                         # Set widget to be Tkinter's Spinbox's

In [10]:
import tkinter as tk
from PIL import Image, ImageTk

class RadioButtonGroup(EntryWidget):

    def __init__(self, root, options, radioButonConfig, default, name, parameterName, showText = True):          # Constructor for RadioButtonGroup class    
        self.__showText = showText                                                                                         # Flag that determines if text will be shown in button  
        super().__init__(root, radioButonConfig, default, name, parameterName)                                             # Constructor for EntryWidget class
        self.__options = options                                                                                           # Options thats determine's the text and val of each button
        self.__photoDict = {option : ImageTk.PhotoImage(file = f"images/{option}.png") for option in options.keys()}       # This dict will contain all the images inside radio button
        
    def execute(self, fill, ipadx, ipady, side, expand):      # This overide's the Widget's execute method 
        for (text, value) in self.__options.items():               # For each pair in the options dict
            self._config["value"] = value                          # Sets value as the val in dict
            self._config["image"] = self.__photoDict[text]         # Sets image for each radio button baeed on dict
            if self.__showText:                                    # If text flag is set to True:
                self._config["text"] = text                             # Add text from options dict, so that it's displayed in radio button
                
            # Create and add the radio button
            RadioButton(self._root).config(self._config).execute(fill = fill, ipadx = ipadx, ipady = ipady, side = side, expand = expand)
        return self

In [11]:
# This class is incharge of getting computer information, like monitor resolution and username

import subprocess
from screeninfo import get_monitors

class System:
    
    # Constructor for System that establishs username, screen resoltuin, and the screen's physical dimensions
    def __init__(self):                                                                                             
        self.__userName = subprocess.check_output(["echo","%username%"], shell=True).decode().replace('\r\n','')   # Find username via CMD (used for PATH)
        self.__screenResolution = [[monitor.width, monitor.height] for monitor in get_monitors()]                  # Find screen resolution for centering windows
        self.__screenSize = [[monitor.width_mm, monitor.height_mm] for monitor in get_monitors()]                  # Get screen physical size 
        
    def getUserName(self):              # Getter for username
        return self.__userName              # return username
    
    def getScreenResolution(self):      # Getter for screen resolution
        return self.__screenResolution      # return the primary screen's resolution
    
    def getScreenSize(self):            # Getter for screen's physical size
        return self.__screenSize            # return screen's physical size

In [12]:
# This class is incharge of interacting with the user preset file called config.json saved in C:\Users\(USERNAME)\AppData\Local\LukeEdit

from os.path import exists
import os
import json

class Config:
    
    def __init__(self):
        self.__userName = System().getUserName()                                               # Find the user's current username
        
        # This will be the default config, if config.json doesn't already exist
        self.__defaultConfig = {"Orientation" : "L",                                           # Orientation of PDF (either 
                                "Vertical" : 25.4,                                             # Margins on the vertical axis
                                "Horizontal" : 25.4,                                           # Margins on the Horizontal axis
                                "OpenLocation" : rf"C:\Users\{self.__userName}\Downloads",     # Default location for opening files
                                "SaveLocation" : rf"C:\Users\{self.__userName}\Documents",     # Default location for saving files
                                "FontSize" : 12}
        
        self.__rootFolder = rf"C:\Users\{self.__userName}\AppData\Local\LukeEdit"              # Root level directory
        self.__configFile = f"{self.__rootFolder}\config.json"                                 # config.json full path
        self.__config = {}                                                                     # raw config data
        self.__checkForConfig()                                                                # Run this to see if a config already exists
        
    def __repr__(self):                                                # Overwrites toString (Java Equivalent) and returns config in stirng form
        return str(self.__config)
    
    def setConfigFile(self):                                           # Write the raw config to the config.json file
        with open(self.__configFile, "w") as file:
            json.dump(self.__config, file)
        return self
        
    def __checkForConfig(self):                                        # This functions checks to see if root directory and config.json 
        if exists(self.__rootFolder) == False:                             # If root doesn't exist
            os.mkdir(self.__rootFolder)                                        # Make root directory
                
            if exists(self.__configFile) == False:                             # If config.json doesn't exist
                self.__config = self.__defaultConfig                               # Set raw config to the default config
                self.setConfigFile()                                               # Create config.json and write to it
        else:
            self.setConfigFromFile()                                       # Else set raw config from config.json
                    
    def __conversionFactor(self, config, factor):                       # Uses to convert margins between millimeters to inches
        for key in config.keys():                                           # For each config
            if key in ["Vertical", "Horizontal"]:                                # If config is a type of margin
                config[key] = round(float(config[key]) * factor,2)                   # Convert margin to either mm: (factor = 25.4) or in: (1 / 25.4) or no change: (factor = 1)
        return config
    
    def getConfig(self):
        return self.__conversionFactor(self.__config, 1)                # Return config, with margins not converted
    
    def setConfig(self, config):                                        # Set raw config to config that is coverted to mm
        self.__config = self.__conversionFactor(config, 25.4)
        return self
                  
    def getConfigFromFile(self):                                        # set raw config from config.json
        with open(self.__configFile) as file:
            config = json.load(file)
        return self.__conversionFactor(config, 1 / 25.4)                # covert the config margins to inches before returning
                    
    def setConfigFromFile(self):                                        # Set raw config based off of config.json
        with open(self.__configFile) as file:
            self.__config = json.load(file)
        return self

In [13]:
# This class is incharge of the displaying the configuartions for PDF generations and intiating the process of generating a new PDF

import tkinter as tk
from PIL import ImageTk, Image
import datetime

class ConfigGUI:
    
    def __init__(self, height, width, parent, content, angle = 0):
        self.__height = height                                                      # Height of window
        self.__width = width                                                        # Width of window
        self.__parent = parent                                                      # Main root window (this window will have to be a child of main root)
        self.__content = content                                                    # Final Text document (to print to PDF)
        self.__angle = angle                                                        # Current angle of spining LukeEdit logo
        
        # Configuartions for all radio buttons
        self.__radioButonConfig = {"font" : ("Arial", 15, 'bold'),                                  # Font family, size, and style
                                   "indicator" : 0,                                                 # Dont show radiobutton icon
                                   "background" : "#B63A00",                                        # bg color
                                   "activebackground" : "light blue",                               # Color when you click button
                                   "compound" : tk.TOP,                                             # Image is on top of text
                                   "selectcolor" : "#0F9724"}                                       # Color of button when selected

        # Configuartions for all spinboxes
        self.__spinboxConfig = {"buttonbackground" : "#F0B408",                                     # button bg color
                                "bg" : "#6B6B6B",                                                   # bg color 
                                "from_" : 0,                                                        # First option
                                "to" : 10,                                                          # Last option
                                "increment" : 0.1,                                                  # Increment for options
                                "justify" : tk.CENTER,                                              # Center text that is displayed
                                "wrap" : True,                                                      # If you go past 0, it goes straight to the max
                                "font" : ('Arial',15,'bold')}                                       # Font family, size, and style

        self.__orientationOptions = {"Portrait" : "P", "Landscape" : "L"}                           # orientation options for radio button
        
        self.__previousConfig = Config().setConfigFromFile().getConfigFromFile()                    # Get config from config.json

        screenResolution = System().getScreenResolution()[0]                                        # Get screen resolution
        
        x = (screenResolution[0] // 2) - (self.__width // 2)                                        # starting x of window
        y = (screenResolution[1] // 2) - (self.__height // 2)                                       # starting y of window

        # Root of window (not main root)
        self.__root = Root("child").config(title = "PDF Configuration",                             # Title of window             
                                           state = tk.NORMAL,                                       # Allow users to affect window
                                           geometry = f"{self.__width}x{self.__height}+{x}+{y}",    # Shape and location of window
                                           ico = r"images/LukeEdit.ico",                            # Change icon
                                           resizable = True).topmost(1).topmost(0).getRoot()        # Aloow window to be resizable

        self.__marignPhotos = {direction : ImageTk.PhotoImage(file = f"images/{direction}.png")     # Create dict to hold photos for margins
                               for direction 
                               in ["Vertical", "Horizontal"]}
        
        self.__config = {}                                                                          # Dict for config data
        
    # Create title portion of window
    def title(self, title):
        Label(self.__root).config({"bg" : "#005595",                                                # bg color
                                   "text" : title,                                                  # text for label
                                   "font" : ("Arial", 25, "bold")}).execute(fill = tk.BOTH,         # set font and add label to window
                                                                            ipadx = 0,
                                                                            ipady = 10,
                                                                            side = tk.TOP,
                                                                            expand = True)
        return self
    
    # Create radio button portion of window for PDF orientation
    def orientation(self):
        orientationFrame = tk.Frame(self.__root, bg = "black")                                                 # Create frame object for orientation radio button portion
        orientationPhotos = {key : f"images/{key}.png" for (key, value) in self.__orientationOptions.items()}  # Dict to hold images

        self.__config["Orientation"] = RadioButtonGroup(root = orientationFrame,                               # Assign root to frame (which is assigned to window)
                                                        options = self.__orientationOptions,                   # This is the key : valur for the radio button
                                                        radioButonConfig = self.__radioButonConfig,            # Use radiobutton configs, defined above
                                                        default = self.__previousConfig["Orientation"],        # From previous config, assign default orientation value
                                                        name = "Orientation",                                  # Name of variable for value
                                                        parameterName = "variable").execute(fill = tk.BOTH,
                                                                                            ipadx = 0,
                                                                                            ipady = 10,
                                                                                            side = tk.RIGHT,
                                                                                            expand = True)
        
        orientationFrame.pack(fill = tk.BOTH, side = tk.RIGHT)                                                 # Add the frame to the window
        return self
        
    # Create spinbox portion of window for PDF margins
    def margin(self, direction):
        marginFrame = tk.Frame(self.__root, bg = "black")                                                     # Create frame object for spinbox margins and it's label portion                        

        Label(marginFrame).config({"bg" : "white",                                                            # bg color
                                   "compound" : tk.TOP,                                                       # Have image shown above text
                                   "image" : self.__marignPhotos[direction],                                  # Get images from image dict
                                   "text" : f"{direction} Margin (in):",                                      # text for label
                                   "font" : ("Arial", 15, "bold")}).execute(fill = tk.BOTH,
                                                                            ipadx = 0,
                                                                            ipady = 10,
                                                                            side = tk.TOP,
                                                                            expand = True)

        self.__config[direction] = Spinbox(root = marginFrame,                                                # Assign root to frame (which is assigned to window)                                                        
                                           spinboxConfig = self.__spinboxConfig,                              # Use spinbox configs, defined above
                                           default = self.__previousConfig[direction],                        # From previous config, assign default margins value
                                           name = direction,                                                  # Name varible based on the type of margin
                                           parameterName = "textvariable").config(self.__spinboxConfig).execute(fill = tk.BOTH,
                                                                                                                ipadx = 0, 
                                                                                                                ipady = 5, 
                                                                                                                side = tk.TOP,
                                                                                                                expand = True)
        
        marginFrame.pack(fill = tk.BOTH, side = tk.RIGHT)                                                    # Add the frame to the window
        return self
    
    # Creates button for starting PDF generation
    def createPDF(self):
        Button(self.__root).config({"bg" : "#E7C000",                                                        # bg color                                     
                                    "text" : "Create PDF",                                                   # Text for button
                                    "command" : self.__runFile,                                              # When button is pressed, start PDF generation process
                                    "font" : ("Arial", 15, 'bold')}).execute(fill = tk.BOTH,
                                                                             ipadx = 0,
                                                                             ipady = 0,
                                                                             side = tk.BOTTOM,
                                                                             expand = True)
        return self
    
    # Writes to config file
    def __getConfig(self):
        
        # For each previous conifg, update 
        for key in self.__config.keys():
            self.__previousConfig[key] = self.__config[key].getValue()   # Assign value to Config object
        Config().setConfig(self.__previousConfig).setConfigFile()        # Write config json to config.json
    
    # Destroy main window and get config
    def __destroy(self):
        self.__getConfig()    # Get the current config for PDF generation
        self.__root.destroy() # Destroy root window
    
    # Starts PDF Process
    def __runFile(self):
        self.__destroy()      # Destroy options window
        self.__saveFile()     # Continue PDF generation process
        
    # Spin the LukeEdit logo
    def __changePhoto(self, label, angleChange):
        self.__angle += angleChange                           # Based on the increment, increase angle
        with Image.open("images/LukeEdit.png") as photo:      # While image is open:
            photo = photo.resize((150, 150))                      # Make image smaller
            photo = photo.rotate(self.__angle)                    # rotate image based off of new angle
        image = ImageTk.PhotoImage(photo)                     # Convert image to ImageTk 
        label.config({"image" : image})                       # Add image to label's config dict
        label.getWidget().image = image                       # Add image to progress window
        
    # Check to see if thread (PDF is still being generated) is still active
    def __isAlive(self, thread, root, label, path, start):
        
        # Provide a msg with duration of process
        label.config({"text" : 
                      f"PDF is being generated\nDuration:  {(datetime.datetime(2000, 1, 1) + (datetime.datetime.now() - start)).strftime('%H:%M:%S')}"})  
        
        # Rotate LukeEdit logo
        angle = self.__changePhoto(label, angleChange = -45)
        
        # If PDF is done generating (thread is dead), destroy window and open PDF
        if not thread.is_alive():
            root.destroy()          # Destroy window
            os.startfile(path)      # Open PDF
        else:
            # Circular reference of this method (start process over again)
            root.after(500, lambda: self.__isAlive(thread, root, label, path, start))
            
    # This creates the progress window, while PDF is being generating
    def __waitingMessage(self, angle, thread, path):
        width = 600                                                                        # Width of window
        height = 200                                                                       # Height of window
        screenResolution = System().getScreenResolution()[0]                               # Get screen resolution
        x = (screenResolution[0] // 2) - (width // 2)                                      # Calculate x (center)
        y = (screenResolution[1] // 2) - (height // 2)                                     # Calculate y (center)

        # Create root for the progress window
        root = Root("parent").config(title = "Test",                                       # Title will not be used (this window will not have top bar
                                     state = tk.NORMAL,                                    # Allow people to interact with window
                                     geometry = f"{width}x{height}+{x}+{y}",               # Controls shape and location of window
                                     ico = r"images/LukeEdit.ico",                         # ico will not be used, window will not have top bar
                                     resizable = True).topmost(1).topmost(0).getRoot()     # Allow window to be resizable 

        root.overrideredirect(1)                                                           # This will eliminate top bar of window
        
        with Image.open("images/LukeEdit.png") as photo:                                   # While image is open:
            photo = photo.resize((150, 150))                                                   # Make image smaller
            photo = photo.rotate(angle)                                                        # rotate image based off of new angle
        image = ImageTk.PhotoImage(photo)                                                  # Convert image to ImageTk 

        # initial label, before progress has been made (lacks duration)
        label = Label(root).config({"text" : "PDF is being generated   ",                  # msg minus duration
                                    "image" : image,                                       # Add image
                                    "font" : ("Arial", 25, "bold"),                        # Control font
                                    "anchor" : tk.CENTER,                                  # Center this label's text
                                    "compound" : tk.RIGHT}).execute(fill = tk.BOTH,        # Make image appear to the right of text
                                                                    ipadx = 0,
                                                                    ipady = 0,
                                                                    side = tk.TOP,
                                                                    expand = True)

        start = datetime.datetime.now()                                                    # This is whent he process started                                           
        root.after(500, lambda: self.__isAlive(thread, root, label, path, start))          # Initial isAlive method use, after this, the method is used via circular
        root.mainloop()                                                                    # Create the progress window
        
    def __saveFile(self):
        filePath = FileDialog().getSaveFilePath(title = "Save", fileTypes = [("pdf file", "*.pdf")])    # Ask for filepath of new PDF
        if filePath:                                                                                    # If filepath is provided:
            document = TextDocument(filePath = filePath, content = self.__content)                          # Create a new TextDocument object with string provided by user
            self.__parent.destroy()                                                                         # Destroy the main GUI application
            angle = 0                                                                                       # Initial angle for LukeEdit logo for progress window
            config = Config().setConfigFromFile()                                                           # Config form PDF, the user chose
            thread = threading.Thread(target = PDF(document, config).createOptimalPDF)                      # Create a thread, dedicated to generating an optimal PDF
            thread.start()                                                                                  # Start this thread
            self.__waitingMessage(angle, thread, filePath)                                                  # Initiate the waiting message (that is defined above)
        
    # Create the main window and bind some commands
    def execute(self):                  
        self.__root.focus_force()                                           # Focus user on this window
        self.__root.protocol("WM_DELETE_WINDOW", self.__runFile)            # If window is destroy via X at the top, run PDF Generation
        self.__root.bind("<Control-Key-p>",lambda event: self.__runFile())  # Bind Ctrl P to run PDF Generation
        self.__root.mainloop()                                              # Create the main window
        
        return self

In [14]:
# This class is used for using the file dialog menu

import tkinter.filedialog as tk_FileDialog

class FileDialog:
    
    def __init__(self):
        self.__previousConfig = Config().setConfigFromFile().getConfigFromFile()                    # This will get previous configs (defualt save and open locations)
    
    # Get open path of file
    def getOpenFilePath(self, title):
        
        #This root object is only used for making the file dialog window the topmost
        rootObject = Root(rootType = "parent").hide().config(title = "Make File Dialog Top Most",   # Not important
                                                      geometry = "",                                # Not important
                                                      ico = r"images/LukeEdit.ico",                 # Not important
                                                      state = tk.NORMAL,                            # Not important
                                                      resizable = False).topmost(1).hide()          # Not important
        
        initialDir = self.__previousConfig["OpenLocation"]                                     # Get the default open location 
        openPath = tk_FileDialog.askopenfilename(initialdir = initialDir,                      # Use the default to set initial opening directory
                                                 title = title)                                # Set title, to let user knowing they are opening a file
        rootObject.show().getRoot().destroy()                                                  # Destroy the root object (not needed anymore)
        return openPath                                                                        # Return either the path or an empty string
    
    # Get path of folder and change the config.json default save or open location
    def getFolderPath(self, label):
        initialDir = self.__previousConfig[label]                                              # Get the default open or save location    
        self.__previousConfig[label] = tk_FileDialog.askdirectory(initialdir = initialDir,     # Use the default to set initial opening or saving  directory
                                                                  title = label[:4])           # title will let user know if they are changing the default open or save location
        if self.__previousConfig[label]:                                                       # If user does select a valid path:
            Config().setConfig(self.__previousConfig).setConfigFile()                               # Change the default open or save location on file in config.json
    
    # Get save path of file
    def getSaveFilePath(self, title, fileTypes):
        initialDir = self.__previousConfig["SaveLocation"]                                     # Get the default save location          
        savePath = tk_FileDialog.asksaveasfilename(default = initialDir,
                                                   initialdir = initialDir,                    # Use the default to set initial saving directory
                                                   title = title,                              # Set title of window "Save"
                                                   filetypes = fileTypes)                      # Set the default file types for saving the file
        return savePath                                                                        # Return the path for the new file or empty string

In [15]:
# This class is incharge of interacting with a string based word document 

from tkinter import messagebox

class TextDocument:
    
    # TextDocument's constructor
    def __init__(self, filePath = "", content = []):
        self.__filePath = filePath                                         # This will be the path of the orignal text document or the new location
        if type(content) == str:                                           # If content is a string datatype:                          
            self.__content = content.split('\n')                               # Convert string to list, split by special chatacter \n (new line char)
            self.__stringOriginally = True                                     # set stringOriginally to True (When object was created, it's content was inially string)
        else:                                                              # Else:
            self.__content = content                                           # Content does not need to be modified 
            self.__stringOriginally = False                                    # set stringOriginally to False (When object was created, it's content was inially list)
        self.__userName = System().getUserName()                           # Find the user's username for path creation
        
    # This override the toString method (Java equiv)
    def __repr__(self):
        return ''.join(self.__content[:50])                                # Show first 50 lines of document
    
    # This function will be used to open a text document and setContent (mehthod)
    def openFile(self):
        self.__filePath = FileDialog().getOpenFilePath(title = "Open")     # Get filepath of content                                 
        if self.__filePath:                                                # If file is chosen
            with open(self.__filePath) as file:                                # while file is open
                try:                                                               # Try to read from file
                    self.__content = file.readlines()                                  # If successful, add contetn from file to content variable
                except UnicodeDecodeError:                                         # If UnicodeDecodeError, provide custom error message (file type not suppored by LukeEdit)
                    self.__filePath = ""                                               # make file path empty
                    
                    # Returm custom error message
                    messagebox.showerror('File Type Error', 'LukeEdit does not support this file type')
                    
        return self
    
    # Get content in either string or list format
    def getContent(self):
        if self.__stringOriginally:
            return '\n'.join(self.__content)                           # If stirng originally join the list by the special character (creating single giant string)
        else:
            return ''.join(self.__content)                             # If not a string (when object was created), there are no special characters "\n" so you join on black ""

    # Find how many lines are in the text document
    def getLength(self):
        return len(self.__content)
    
    # Find the max number of chars per line (for the entire document)
    def getMaxChar(self):
        return max([len(line) for line in self.__content])
    
    # Return the last part of the filepath (the file's name)
    def getFileName(self):
        return self.__filePath.split("/")[-1]
    
    # Return full file path (associated with the text document)
    def getFilePath(self):
        return self.__filePath
    
    # Return the raw content (in the form of a list of strings (which represent each line of the document))
    def getContentList(self):
        return self.__content
    
    # This will return the index of the line (where the specific case insensitive target was found
    def findTarget(self, target):
        if target:                                                    # If target is not an empty string
            i = 0                                                     # var to hold current index (line)
            linesFound = []                                           # Hold all lines, where target is found                    
            for line in [line.lower() for line in self.__content]:    # For each line in a lower case form of the raw document
                if line.find(target.lower()) != -1:                       # If lower case target is found in lower case line
                    linesFound.append(i)                                      # Add the line index to the final list
                i += 1                                                    # Move on to next line

            return linesFound                                         # Return the find list 

In [16]:
# This class is a wrapper for tkinter's Menu object

import tkinter as tk

class Menu:
    
    # Constructor 
    def __init__(self, root):
        self.__menuBar = tk.Menu(root)                                                            # This is the entire menu object                                          
        self.__menuTabs = {}                                                                      # This dict will hold allt he tabs, within the menu (File, Edit, etc.)
        root.config(menu = self.__menuBar)                                                        # Add the menu object to the main root window

    # Attach tabs to menu object
    def addTab(self, *tabs):
        for tab in tabs:                                                                          # For each tab in the tab's list
            self.__menuTabs[tab] = tk.Menu(self.__menuBar, tearoff = 0)                                # Add a menu object to the tab dict
            self.__menuBar.add_cascade(label = tab, menu = self.__menuTabs[tab], underline = 0)        # Add the menu object, to the parent menu object
        return self
    
    # Assign a tab a specifc command and label (like Save)
    def addCommand(self, tab, label, config, command = None):
        if command:                                             # If command is not None
            config["command"] = command                             # Assign a command to a label
        config["label"] = label                                 # Assign a label to the command
        self.__menuTabs[tab].add_command(config)                # Add this command to the tab
        return self

In [17]:
# This class creates a PDF and gathers information on a PDF document

import fpdf
import pdfplumber

class PDF:
    
    # Constructor
    def __init__(self, document, config):
        self.__document = document                                                 # Raw String, that will be added to PDF
        self.__filePath = document.getFilePath()                                   # Save location for PDF
        self.__config = config                                                     # Config object that contains settings for PDF document like (margins, orientation, etc.)
 
    # Create FPDF object and gettting the configs from the Config object
    def __configPDF(self):
        pdf = fpdf.FPDF(format = 'letter', 
                        orientation = self.__config.getConfig()["Orientation"])      # Create FPDF and assign orientation
        
        pdf.set_margins(left = self.__config.getConfig()["Horizontal"],              # Assign left (Horizontal Marign)
                        top = self.__config.getConfig()["Vertical"])                 # Assign right (Vertical Marign)
        
        pdf.set_auto_page_break(auto = True,                                         # Auto page break on
                                margin = self.__config.getConfig()["Vertical"])      # Assign bottom (Vertical Marign)                                
        pdf.add_page()                                                               # Create a new page for PDF (Required before creating a PDF)
        return pdf                                                                   # Return FPDF object

    # Create the PDF document
    def __createPdf(self, fontSize, text):
        pdf = self.__configPDF()                       # Configure PDF document
        pdf.set_font("Courier", size = fontSize)       # Set font and font size
        pdf.multi_cell(w = 0, h = 5, txt = text)       # Add text to PDF document
        pdf.output(self.__filePath)                    # Create and save PDF document

    # Find the number of characters, on the first line of the first page of the PDF ducment created
    def __getCharLenPDF(self):
        with pdfplumber.open(self.__filePath) as pdfFile:                 # While PDF document is opened
            return len(pdfFile.pages[0].extract_text().split('\n')[0])        # From the first page, convert string to list and get first line's length

    # Find the optimal font size, for the PDF document
    def createOptimalPDF(self):
        maxChar = self.__document.getMaxChar()                                          # Find the max chars per line of raw text document
        increment = 0.1                                                                 # Each iter of the algorithm will be incremented by this number
        fontSize = increment                                                            # This will be the smallest font size used
        while True:                                                                     # Run algorithm loop until optimal font size is found
            self.__createPdf(fontSize, "O" * maxChar)                                       # Create a test PDF with just "O"'s
            currentCharLen = self.__getCharLenPDF()                                         # Then find out if this fontsize caused the line to break
            
            if currentCharLen < maxChar:                                                    # If the line broke
                self.__createPdf(fontSize - increment, self.__document.getContent())             # Create the actual final PDF, with raw text and the optimal font size
                break                                                                            # Break out of the loop
                
            fontSize += increment                                                           # Go to the next font size to test

In [18]:
# This class will allow the user to moddify the position of the text displayed in the text widget

class Justify:
    
    # Constructor
    def __init__(self, root, textWidget):
        self.__textWidget = textWidget                                                                   # This will be the text object
        radioButonConfig = {"command" : self.test,                                                       # This command will change the postion of text displayed using the poorly named test method
                            "indicator" : 0,                                                             # This will make radiobutton symbol disapear
                            "background" : "dark grey",                                                  # bg color
                            "activebackground" : "light blue",                                           # Actove bg color
                            "compound" : tk.TOP,                                                         # Show image above text
                            "selectcolor" : "light grey"}                                                # select color
        
        orientationOptions = {"Right" : "Right", "Center" : "Center", "Left" : "Left"}                   # Values for the radio button group
        orientationPhotos = {key : f"images/{key}.png" for (key, value) in orientationOptions.items()}   # Images that will relate to each choice

        self.__RGB = RadioButtonGroup(root = root,                                                       # Root where the RGB will  be displayed
                                      options = orientationOptions,                                      # These are options the user can chose
                                      radioButonConfig = radioButonConfig,                               # Use options defined above
                                      default = "Center",                                                # Default option (Center)
                                      name = "Orientation",                                              # Variable name
                                      parameterName = "variable",                                        # Name of the tkinter option
                                      showText = False).execute(fill = tk.BOTH,
                                                                ipadx = 0,
                                                                ipady = 0,
                                                                side = tk.RIGHT,
                                                                expand = True)
    # This command will change the postion of text displayed using the poorly named test method
    def test(self):
        justifyValue = self.__RGB.getValue()                                                             # Get the value of what the user chose
        self.__textWidget.tag_configure("center", justify = justifyValue.lower())                        # Create tag that will center the text displayed
        self.__textWidget.tag_add("center", "1.0", "end")                                                # Add this tag to widget

In [19]:
# This class is incharge of all the commands and keybinding (the user is allowed to do)

class Controller:
    
    # Constructor
    def __init__(self, rootObject, text, entry, findCountLabel, fontSizeLabel, modeLabel):
        self.__rootObject = rootObject                                                        # This is the main application window class object
        self.__root = self.__rootObject.getRoot()                                             # This is the main application window                              
        self.__text = text                                                                    # This is the wrapper class object for tkinter's textscolled wdiget
        self.__entry = entry                                                                  # This is the wrapper class object for tkinter's Entry wdiget
        self.__findCountLabel = findCountLabel                                                # This label will either display nothing, the number of results found, or an error message
        self.__fontSizeLabel = fontSizeLabel                                                  # This will show the font size of the text information displayed in the Text widget
        self.__modeLabel = modeLabel                                                          # This will display either View Mode or Edit Mode
        self.__currentConfig = Config().setConfigFromFile().getConfigFromFile()               # Read config from config.json
        self.__fontSizeLabel.config({"text" : f"{self.__currentConfig['FontSize']} pts"})     # Initialize font size based on config.json
        self.__viewMode()                                                                     # Set application to view mode by default
        
    # This is the settings and keybindings for view mode
    def __viewMode(self):
        self.__entry.getWidget().focus_force()                                                     # Focus user cursor on the entry widget for finding queries                                                                                
        self.__modeLabel.config({"text" : "- View Mode"})                                          # Change the mode label to View Mode
        self.__text.config({"cursor" : "dotbox",                                                   # Change cursor symbol to dotbox (Making it easier to select lines)
                            "selectbackground" : "black",                                          # Eliminate the ability for people to highlights parts of lines
                            "state" : tk.DISABLED})                                                # Disbale the user's ability to edit text that is displayed in the Text widget
        self.__root.bind("<Escape>",                 lambda event: self.__root.destroy())          # When user clicks escape, they will destory the Luke Edit Application
        self.__root.bind("<Button-1>",               lambda event: self.__text.makeSelection())    # When a user left clicks, they can select either the first or last line of selection
        self.__entry.getWidget().bind("<Button-1>",  lambda event: self.__resetSelection())        # When a user right clicks on the entry widget, they can reset everything  
        self.__root.bind("<Button-3>",               lambda event: self.__resetSelection())        # When a user right clicks, they can reset everything (Word searched, lines selected)
        self.__root.bind("<Control-Key-z>",          lambda event: self.__resetSelection())        # When a user clicks Ctrl-Z, they can reset everything (Word searched, lines selected)
        self.__root.bind("<Return>",                 lambda event: self.__queryFile(1))            # When a user clicks Enter, start query for word going from top to bottom
        self.__root.bind("<Left>",                   lambda event: self.__queryFile(-1))           # When a user clicks Left Arrow, start query for word going from bottom to top
        self.__root.bind("<Right>",                  lambda event: self.__queryFile(1))            # When a user clicks Right Arrow, start query for word going from top to bottom
        self.__root.bind("<Control-Key-p>",          lambda event: self.__createPDF())             # When a user clicks Ctrl-P, The lines current selected will be used and the PDF creation will cont
        self.__root.bind("<Control-Key-s>",          lambda event: self.__saveFile())              # When a user clicks Ctrl-S, The lines current selected will be saved as a new txt doc
        self.__text.getWidget().bind("<Motion>",     lambda event: self.__text.highlightLine())    # When a user moves mouse, the line where mouse is over will  be highlighted
        self.__root.bind("<Control-minus>",          lambda event, fontSizeDirection = -1: self.__changeFontSize(event, fontSizeDirection)) # Decrease font size of text displayed in text widget
        self.__root.bind("<Control-equal>",          lambda event, fontSizeDirection = 1: self.__changeFontSize(event, fontSizeDirection))  # Decrease font size of text displayed in text widget
        self.__root.bind("<Control-Key-e>",          lambda event: self.__editMode())              # When a user clicks Ctrl-E, go to Edit Mode
        
    # This is the settings and keybindings for edit mode
    def __editMode(self):
        self.__modeLabel.config({"text" : "- Edit Mode"})                           # Change the mode label to Edit Mode                      
        self.__resetSelection()                                                     # Reset to default state
        self.__text.getWidget().focus_force()                                       # Focus user cursor on the entry widget for finding queries      
        self.__entry.getWidget().unbind("<Button-1>")                               # Unbind the ability for a user to reset everything
        self.__text.config({"cursor" : "xterm",                                     # Change cursor to the one most common for text editing
                            "selectbackground" : "lightblue",                       # Chnage the select color
                            "state" : tk.NORMAL,                                    # Enable eiditng of the content within the Text widget
                            "insertbackground" : "white"})                          # change the insert background to white
        self.__root.unbind("<Button-1>")                                            
        self.__root.unbind("<Button-3>")                                            # Unbind the ability for a user to reset everything
        self.__root.unbind("<Control-Key-z>")                                       # Unbind the ability for a user to reset everything
        self.__root.unbind("<Return>")                                              # Unbind the ability for a user to comb through a searched result
        self.__root.unbind("<Left>")                                                # Unbind the ability for a user to comb through a searched result
        self.__root.unbind("<Right>")                                               # Unbind the ability for a user to comb through a searched result
        self.__root.unbind("<Control-Key-p>")                                       # Unbind the ability for a user to create a PDF based off a selection
        self.__root.unbind("<Control-Key-s>")                                       # Unbind the ability for a user to save a file bassed off a slection
        self.__text.getWidget().unbind("<Motion>")                                  # Unbind the auto highlighting caused by mouse movement
        self.__root.bind("<Control-Key-e>", lambda event: self.__viewMode())        # Go back to view mode (allowing for the toggling of modes)

    # This method's sole focus, is reseting LukeEdit back to it's default state
    def __resetSelection(self):
        self.__text.getWidget().tag_delete("selection")                             # Remove any highlighted sections
        self.__text.getWidget().tag_delete("select")                                # Remove any hightlighted lines
        self.__text.getWidget().tag_delete("find")                                  # Remove any highlighted found words
        self.__text.getWidget().tag_delete("highlight")                             # Remove any lines that used the Text object's highlightLine method
        self.__findCountLabel.getWidget().config(text = "")                         # Remove any text within the findCountLabel
        self.__entry.getWidget().delete(0, tk.END)                                  # Remove any text entered within the Enrty widget
        self.__entry.getWidget().focus_force()                                      # Refocus user's cursor back to the Entry widget object
        self.__text.resetSelectedPoints()                                           # Reset any lines that were selected and ready to be converted to txt or pdf
        
    # This method starts the process of find a sopecfic word, that the user entered into the Entry widget
    def __queryFile(self, direction): 
        result = self.__entry.getInput()                         # Get the information the user entered
        if result:                                               # If user did in fact, enter text
            count = self.__text.queryFile(result, direction)          # Utilize the text objects queryFile (which utilizes the TextDocuments's find feature), which has spits out a num of found items
            self.__findCountLabel.config({"text" : count})            # Show to the user, the number of found items, and whihc item is currently highlighted
    
    # This method starts the process of converting a user's selction of lines, into a properly formatted PDF document
    def __createPDF(self):
        # If the user selected muluple lines
        if self.__text.getWidget().tag_ranges("selection"):
            # Get the user's selected lines
            content = self.__text.getWidget().get("selection.first", "selection.last")
            # And give the user the PDF configuration window (so they can choose how the PDF should be formatted)
            ConfigGUI(height = 400, width = 900, parent = self.__root, content = content).title("PDF Configuration").createPDF().orientation().margin("Vertical").margin("Horizontal").execute()
        # If the user did not slelect any lines, provide them a simple error message
        else:
            self.__findCountLabel.config({"text" : "No Lines Selected"})
    # This method saves a user's selction of lines, into a new text document
    def __saveFile(self):
        filePath = FileDialog().getSaveFilePath(title = "Save", fileTypes = [("text file", "*.txt")]) # Ask the user for the location, of this new texdt document
        if filePath:                                                                                  # If the user did in fact chose a location
            content = self.__text.getWidget().get("selection.first","selection.last")                     # Get the user's selected lines
            with open(filePath, 'w') as file:                                                             # While the new empty file is created and open
                file.write(content)                                                                           # Write selected lines to file 
            self.__root.destroy()                                                                         # Destroy LukeEdit application
            os.startfile(filePath)                                                                        # Open new text document
    
    # Increase or decrease the font size of the text displayed in the Text widget 
    def __changeFontSize(self, event, fontSizeDirection):
        previousConfig = Config().setConfigFromFile().getConfigFromFile()                             # Get the configs from the config.json file
        currentFontFamily = self.__text.getWidget().cget('font').split(' ')[0]                        # Find out what the current font is
        currentFontSize = int(self.__text.getWidget().cget('font').split(' ')[-1])                    # Find only what the font size is                     
        newFontSize = currentFontSize + fontSizeDirection                                             # Increase or decrease the current font size
        self.__text.getWidget().config(font = (currentFontFamily, newFontSize))                       # Add the new font size to the Text widget
        #self.__entry.getWidget().config(font = (currentFontFamily, newFontSize))
        previousConfig["FontSize"] = newFontSize
        self.__fontSizeLabel.config({"text" : f"{newFontSize} pts"})                                  # Display what the new font size is 
        Config().setConfig(previousConfig).setConfigFile()                                            # Add this new font size to the config.json
        
    # This uses recursion to create a new instance of LukeEdit
    def newFile(self, root, main):
        self.__root.destroy()                                    # Destroy the current Luke Edit instance
        main()                                                   # Create a new instance of LukeEdit

In [30]:
# LukeEdit application
def main():
    document = TextDocument().openFile()                                                                     # Ask the user to find a new file and then open it and get content              
    if document.getFilePath():                                                                               # If user selects a file:
        rootObject = Root("parent").hide()                                                                       # Create a root object for the main window and hide it
        root = rootObject.getRoot()                                                                              # Assign Tkinter's root object to root var
        
        # This will hold the configs for the text widget 
        textConfig = {"fg" : "white",                                                                            # fg color                                                                    
                      "bg" : "black",                                                                            # bg color
                      "height" : document.getLength(),                                                           # determine the height of text widget by finding num of lines
                      "width" : document.getMaxChar(),                                                           # determine the width of text widget by finding max char per line
                      "cursor" : "dotbox",                                                                       # change cursor to the default for view mode
                      "state" : tk.DISABLED,                                                                     # change state to default for view mode
                      "selectbackground" : "black",                                                              # vide mode requires a select bg of black
                      "padx" : 0,                                                                                # padding of x axis
                      "pady" : 0,                                                                                # padding of y axis
                      "undo" : True}                                                                             # Aloow the user to ctrl z, when they make mistake
        
        # config for interactive menu bar
        menuButtonConfig = {"state" : tk.NORMAL,                                                                 # Make it so user can interact with it                                   
                            "font" : ("Courier", 8)}                                                             # Font style
        
        # config for non-interactive menu bar
        menuLabelConfig = {"state" : tk.DISABLED,                                                                # Make it so user cannot interact with it    
                           "font" : ("Courier", 8)}                                                              # Font style
            
        menu = Menu(root).addTab("File", "Edit", "Settings")                                                     # These will be the tabs for menu
        
        findFrame = tk.Frame(root)                                                                               # Frame for widgets that are displayed above Tkinters Text widget
        
        # Used to display the count of words found 
        findCountLabel = Label(findFrame).config({"text" : "",                                                   # Since nothing was found yet, default to blank string
                                                  "font" : ("Arial", 12, "bold")}).execute(fill = tk.BOTH,       
                                                                                           ipadx = 5,
                                                                                           ipady = 0,
                                                                                           side = tk.LEFT,
                                                                                           expand = True)
        
         # Used to allow user to enter a word, that they want to find
        entry = Entry(findFrame).config({"font" : ("Arial", 15)}).execute(fill = tk.Y,
                                                                          ipadx = 0,
                                                                          ipady = 0,
                                                                          side = tk.LEFT,
                                                                          expand = False)
        # Open the magnifying glass symbol and resize it
        with Image.open("images/zoom.png") as photo:
            photo = photo.resize((25, 25))
            
        image = ImageTk.PhotoImage(photo)                                                                       # Convert the symbol to Tkinter's ImagerTk file
        
        fontSizeLabel = Label(findFrame).config({"text" : "",                                                   # Used to display the current font size of text within the text widget, blank first
                                                 "font" : ("Arial", 12, "bold"),                                # Font of label
                                                 "image" : image,                                               # Show magnifying glass symbol to the left (closer to the entry object)
                                                 "compound" : tk.LEFT}).execute(fill = tk.BOTH,
                                                                                 ipadx = 0,
                                                                                 ipady = 0,
                                                                                 side = tk.LEFT,
                                                                                 expand = True)
        # This will show which mode Luke Edit is in
        modeLabel = Label(findFrame).config({"text" : "",
                                             "font" : ("Arial", 12, "bold")}).execute(fill = tk.BOTH,
                                                                                      ipadx = 10,
                                                                                      ipady = 0,
                                                                                      side = tk.LEFT,
                                                                                      expand = True)
        
         # Add all these items to the main window
        findFrame.pack(side = tk.TOP) 
        
        # Add the text widget to the main window (below all the other stuff)
        text = Text(root, document = document).config(textConfig).execute(fill = tk.BOTH,
                                                                          ipadx = 0,
                                                                          ipady = 0,
                                                                          side = tk.TOP,
                                                                          expand = True)
        
         # Add a radio group that allows the user to moddify the justify of the text
        Justify(root = findFrame, textWidget = text.getWidget())

 
        
        
        
        # Create the controller class with all the user interactive functionality
        controller = Controller(rootObject = rootObject, 
                                text = text, 
                                entry = entry, 
                                findCountLabel = findCountLabel, 
                                fontSizeLabel = fontSizeLabel, 
                                modeLabel = modeLabel)
        
        # All the menu options (which are self explaintory) and I'm too lazy to continue commenting
        menu.addCommand("File", "New File", menuButtonConfig, lambda: controller.newFile(root, main))
        
        menu.addCommand("Edit", "Create PDF                       Ctrl+P", menuLabelConfig)
        menu.addCommand("Edit", "Save File                        Ctrl+S", menuLabelConfig)
        menu.addCommand("Edit", "Increase Font Size            Ctrl+Plus", menuLabelConfig)
        menu.addCommand("Edit", "Decrease Font Size           Ctrl+Minus", menuLabelConfig)
        menu.addCommand("Edit", "Query Text Down             Right Arrow", menuLabelConfig)
        menu.addCommand("Edit", "Query Text Down                   Enter", menuLabelConfig)
        menu.addCommand("Edit", "Query Text Up                Left Arrow", menuLabelConfig)
        menu.addCommand("Edit", "Line Selection        Left Mouse Button", menuLabelConfig)
        menu.addCommand("Edit", "Reset Selection      Right Mouse Button", menuLabelConfig)
        menu.addCommand("Edit", "Reset Selection                  Ctrl+Z", menuLabelConfig)
            
        menu.addCommand("Settings", "Save Location", menuButtonConfig, lambda: FileDialog().getFolderPath(label = "SaveLocation"))
        menu.addCommand("Settings", "Open Location", menuButtonConfig, lambda: FileDialog().getFolderPath(label = "OpenLocation"))  
        
        # Show the root object finally 
        rootObject.show().config(title = document.getFileName(),          # Title is determined by the orignal file's name 
                                 geometry = "",                           # Geometry is not required due to zoomed state
                                 ico = r"images/LukeEdit.ico",            # Add the custom LukeEdit logo
                                 state = "zoomed",                        # Makes the application fullscreen except for the Windows 10 toolbar
                                 resizable = True).topmost(1).topmost(0)  # Allow the main window to be reizable and makeit the top most window for a second
        
        root.mainloop()                                                   # Run the main window of LukeEdit
test = main()                                                             # Execute LukeEdit application
print("DONE")

DONE


In [21]:
# 26 minutes to create 24,870 pages.... Almost a 1000 pages per min
#self.__filePath = '/'.join(self.__filePath.split('/')[:-1]) + "/" + str(fontSize) + ".pdf"